# Run fitsverify

In [1]:
import os
import sys
import re
import shutil
import subprocess as sp
from configparser import ConfigParser
from random import choice, choices
specprod = 'guadalupe'
specprod_path = os.path.join(os.environ['DESI_SPECTRO_REDUX'], specprod)

## Create input file

In [2]:
files = os.path.join(os.environ['CSCRATCH'], f'{specprod}_files.txt')
if not os.path.exists(files):
    # os.chdir(specprod_path)
    with open(files, 'w') as out:
        command = ['find', 'calibnight', 'exposures', 'healpix', 'preproc', 'tiles', 'zcatalog', '-type', 'f']
        proc = sp.Popen(command, stdout=out, stderr=sp.DEVNULL, cwd=specprod_path)
        status = proc.wait()
        for k in ('exposures', 'tiles'):
            for l in ('fits', 'csv'):
                if os.path.exists(os.path.join(specprod_path, f'{k}-{specprod}.{l}')):
                    out.write(f'{k}-{specprod}.{l}\n')

## List of Regular Expressions

In [6]:
parser = ConfigParser()
parser.read_string("""
[top]
exposures = exposures-[a-z]+\.fits;exposures-SPECPROD.rst
tiles = tiles-[a-z]+\.fits;tiles-SPECPROD.rst

[calibnight]
biasnight = calibnight/[0-9]{8}/biasnight-[brz][0-9]-[0-9]{8}\.fits\.gz;calibnight/NIGHT/biasnight-CAMERA-NIGHT.rst
biasnighttest = calibnight/[0-9]{8}/biasnight-[brz][0-9]-[0-9]{8}\.fits\.gz;calibnight/NIGHT/biasnighttest-CAMERA-NIGHT.rst
fiberflatnight = calibnight/[0-9]{8}/fiberflatnight-[brz][0-9]-[0-9]{8}\.fits;calibnight/NIGHT/fiberflatnight-CAMERA-NIGHT.rst
psfnight = calibnight/[0-9]{8}/psfnight-[brz][0-9]-[0-9]{8}\.fits;calibnight/NIGHT/psfnight-CAMERA-NIGHT.rst

[exposures]
cframe = exposures/[0-9]{8}/[0-9]{8}/cframe-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/cframe-CAMERA-EXPID.rst
exposure-qa = exposures/[0-9]{8}/[0-9]{8}/exposure-qa-[0-9]{8}\.fits;exposures/NIGHT/EXPID/exposure-qa-EXPID.rst
fiberflat = exposures/[0-9]{8}/[0-9]{8}/fiberflat-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fiberflat-CAMERA-EXPID.rst
fiberflatexp = exposures/[0-9]{8}/[0-9]{8}/fiberflatexp-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fiberflatexp-CAMERA-EXPID.rst
fit-psf = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-CAMERA-EXPID.rst
fit-psf-before-listed = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-listed-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-before-blacklisted-CAMERA-EXPID.rst
fit-psf-before-listed-fix = exposures/[0-9]{8}/[0-9]{8}/fit-psf-before-listed-fix-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-before-blacklisted-fix-CAMERA-EXPID.rst
fit-psf-fixed-listed = exposures/[0-9]{8}/[0-9]{8}/fit-psf-fixed-listed-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fit-psf-fixed-blacklisted-CAMERA-EXPID.rst
fluxcalib = exposures/[0-9]{8}/[0-9]{8}/fluxcalib-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/fluxcalib-CAMERA-EXPID.rst
frame = exposures/[0-9]{8}/[0-9]{8}/frame-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/frame-CAMERA-EXPID.rst
psf = exposures/[0-9]{8}/[0-9]{8}/psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/psf-CAMERA-EXPID.rst
sframe = exposures/[0-9]{8}/[0-9]{8}/sframe-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/sframe-CAMERA-EXPID.rst
shifted-input-psf = exposures/[0-9]{8}/[0-9]{8}/shifted-input-psf-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/shifted-input-psf-CAMERA-EXPID.rst
sky = exposures/[0-9]{8}/[0-9]{8}/sky-[brz][0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/sky-CAMERA-EXPID.rst
stdstars = exposures/[0-9]{8}/[0-9]{8}/stdstars-[0-9]-[0-9]{8}\.fits;exposures/NIGHT/EXPID/stdstars-SPECTROGRAPH-EXPID.rst

[healpix]
coadd = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/coadd-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/coadd-SURVEY-PROGRAM-PIXNUM.rst
emline = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/emline-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/coadd-SURVEY-PROGRAM-PIXNUM.rst
qso_mgii = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_mgii-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/qso_mgii-SURVEY-PROGRAM-PIXNUM.rst
qso_qn = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/qso_qn-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/qso_qn-SURVEY-PROGRAM-PIXNUM.rst
redrock = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/redrock-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/redrock-SURVEY-PROGRAM-PIXNUM.rst
spectra = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/spectra-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.fits;healpix/SURVEY/PROGRAM/PIXGROUP/PIXNUM/spectra-SURVEY-PROGRAM-PIXNUM.rst
tilepix = healpix/tilepix\.fits;healpix/tilepix.rst

[preproc]
fibermap = preproc/[0-9]{8}/[0-9]{8}/fibermap-[0-9]{8}\.fits;preproc/NIGHT/EXPID/fibermap-EXPID.rst
preproc = preproc/[0-9]{8}/[0-9]{8}/preproc-[brz][0-9]-[0-9]{8}\.fits;preproc/NIGHT/EXPID/preproc-CAMERA-EXPID.rst

[tiles]
coadd = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/coadd-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/coadd-SPECTROGRAPH-NIGHT-EXPID.rst
emline = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/emline-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/emline-SPECTROGRAPH-NIGHT-EXPID.rst
qso_mgii = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_mgii-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/qso_mgii-SPECTROGRAPH-NIGHT-EXPID.rst
qso_qn = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/qso_qn-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/qso_qn-SPECTROGRAPH-NIGHT-EXPID.rst
redrock = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/redrock-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/redrock-SPECTROGRAPH-NIGHT-EXPID.rst
spectra = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/spectra-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/spectra-SPECTROGRAPH-NIGHT-EXPID.rst
tile-qa = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/tile-qa-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/tile-qa-NIGHT-EXPID.rst
zmtl = tiles/(cumulative|pernight)/[0-9]+/[0-9]{8}/zmtl-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.fits;tiles/TILETYPE/TILEID/NIGHT/tile-qa-NIGHT-EXPID.rst

[tiles:depth]
coadd = tiles/[14]x_depth/[0-9]+/[0-9]/coadd-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/coadd-SPECTROGRAPH-TILEID-EXPID.rst
emline = tiles/[14]x_depth/[0-9]+/[0-9]/emline-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/emline-SPECTROGRAPH-TILEID-EXPID.rst
qso_mgii = tiles/[14]x_depth/[0-9]+/[0-9]/qso_mgii-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_mgii-SPECTROGRAPH-TILEID-EXPID.rst
qso_qn = tiles/[14]x_depth/[0-9]+/[0-9]/qso_qn-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_sn-SPECTROGRAPH-TILEID-EXPID.rst
redrock = tiles/[14]x_depth/[0-9]+/[0-9]/redrock-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/redrock-SPECTROGRAPH-TILEID-EXPID.rst
spectra = tiles/[14]x_depth/[0-9]+/[0-9]/spectra-[0-9]-[0-9]+-[14]xsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/spectra-SPECTROGRAPH-TILEID-EXPID.rst

[tiles:lowspeed]
coadd = tiles/lowspeed/[0-9]+/[0-9]/coadd-[0-9]-[0-9]+-lowspeedsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/coadd-SPECTROGRAPH-TILEID-EXPID.rst
emline = tiles/lowspeed/[0-9]+/[0-9]/emline-[0-9]-[0-9]+-lowspeedsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/emline-SPECTROGRAPH-TILEID-EXPID.rst
qso_mgii = tiles/lowspeed/[0-9]+/[0-9]/qso_mgii-[0-9]-[0-9]+-lowspeedsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_mgii-SPECTROGRAPH-TILEID-EXPID.rst
qso_qn = tiles/lowspeed/[0-9]+/[0-9]/qso_qn-[0-9]-[0-9]+-lowspeedsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/qso_sn-SPECTROGRAPH-TILEID-EXPID.rst
redrock = tiles/lowspeed/[0-9]+/[0-9]/redrock-[0-9]-[0-9]+-lowspeedsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/redrock-SPECTROGRAPH-TILEID-EXPID.rst
spectra = tiles/lowspeed/[0-9]+/[0-9]/spectra-[0-9]-[0-9]+-lowspeedsubset[1-6]\.fits;tiles/DEPTH/TILEID/SPECTROGRAPH/spectra-SPECTROGRAPH-TILEID-EXPID.rst

[zcatalog]
zpix = zcatalog/zpix-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)\.fits;zcatalog/zpix-SURVEY-PROGRAM.rst
ztile = zcatalog/ztile-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-(cumulative|perexp|pernight)\.fits;zcatalog/ztile-SURVEY-PROGRAM-TILETYPE.rst

[exclude]
fit-psf = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}_[0-9][0-9]\.fits;dummy

[other]
badcolumns = calibnight/[0-9]{8}/badcolumns-[brz][0-9]-[0-9]{8}\.csv;dummy
calibstars = exposures/[0-9]{8}/[0-9]{8}/calibstars-[0-9]{8}\.csv;dummy
rrdetails = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/rrdetails-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.h5;dummy
hpixexp = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/hpixexp-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.csv;dummy
healpixlog = healpix/(cmx|sv1|sv2|sv3|main|special)/(backup|bright|dark|other)/[0-9]+/[0-9]+/logs/(coadd|emline|qso_mgii|qso_qn|redrock|spectra)-(cmx|sv1|sv2|sv3|main|special)-(backup|bright|dark|other)-[0-9]+\.log;dummy
healpixjson = healpix/tilepix\.json;dummy
tilepix = preproc/[0-9]{8}/[0-9]{8}/tilepix-[0-9]+\.json;dummy
tileslog = tiles/(cumulative|perexp|pernight|1xdepth)/[0-9]+/[0-9]{8}/logs/(coadd|emline|qso_mgii|qso_qn|redrock|spectra|zmtl)-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.log;dummy
tileqalog = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/logs/tile-qa-[0-9]+-(thru|exp|)[0-9]{8}\.log;dummy
tilesdetails = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/rrdetails-[0-9]-[0-9]+-(thru|exp|)[0-9]{8}\.h5;dummy
depthdetails = tiles/[14]x_depth/[0-9]+/[0-9]/rrdetails-[0-9]-[0-9]+-[14]xsubset[1-6]\.h5;dummy
depthlog = tiles/[14]x_depth/[0-9]+/[0-9]/logs/(coadd|emline|qso_mgii|qso_qn|redrock|spectra|zmtl)-[0-9]-[0-9]+-[14]xsubset[1-6]\.log;dummy
speeddetails = tiles/lowspeed/[0-9]+/[0-9]/rrdetails-[0-9]-[0-9]+-lowspeedsubset[1-6]\.h5;dummy
speedlog = tiles/lowspeed/[0-9]+/[0-9]/logs/(coadd|emline|qso_mgii|qso_qn|redrock|spectra|zmtl)-[0-9]-[0-9]+-lowspeedsubset[1-6]\.log;dummy
tile-qa = tiles/(cumulative|perexp|pernight)/[0-9]+/[0-9]{8}/tile-qa-[0-9]+-(thru|exp|)[0-9]{8}\.png;dummy
tiles = tiles-[a-z]+\.csv;dummy
exposures = exposures-[a-z]+\.csv;dummy
"""
    )

# [exclude]
# calibnight = calibnight/[0-9]{8}/tmp/*;dummy
# exposures = exposures/[0-9]{8}/old/*;dummy
# frame = exposures/[0-9]{8}/[0-9]{8}/frame-[brz][0-9]-[0-9]{8}-no-badcolumn-mask\.fits;dummy
# fit = exposures/[0-9]{8}/[0-9]{8}/fit-psf-[brz][0-9]-[0-9]{8}_[0-9][0-9]\.fits;dummy
# preproc = preproc/[0-9]{8}/old/*;dummy
# redrock = tiles/pernight/[0-9]+/[0-9]{8}/redrock-tmp-[0-9]-[0-9]+-[0-9]{8}\.fits;dummy
# rrdetails = tiles/pernight/[0-9]+/[0-9]{8}/rrdetails-tmp-[0-9]-[0-9]+-[0-9]{8}\.fits;dummy


## Precompile Regular Expressions

In [7]:
r = dict()
structure = dict()
for s in parser.sections():
    r[s] = dict()
    structure[s] = dict()
    for key, value in parser.items(s):
        v = value.split(';')
        r[s][key] = re.compile(v[0])
        structure[s][key] = v[1]

## Scan the list of files

In [8]:
with open(files) as e:
    data = e.readlines()
scanable = dict()
for file in data:
    ff = file.strip()
    f = ff.split('/')
    fits = False
    if ff.endswith('.fits') or ff.endswith('.fits.gz'):
        fits = True
    if fits:
        if len(f) == 1:
            section = 'top'
        elif f[1] == '1x_depth' or f[1] == '4x_depth':
            section = 'tiles:depth'
        elif f[1] == 'lowspeed':
            section = 'tiles:lowspeed'
        else:
            section = f[0]
    else:
        section = 'other'
    if fits and section not in scanable:
        scanable[section] = dict()
    excluded = False
    for key in r['exclude']:
        m = r['exclude'][key].match(ff)
        if m is not None:
            excluded = True
    if excluded:
        continue
    matched = False
    for key in r[section]:
        m = r[section][key].match(ff)
        if m is not None:
            matched = True
            if fits:
                if key in scanable[section]:
                    scanable[section][key].append(ff)
                else:
                    scanable[section][key] = [ff]
    if not matched:
        print("ERROR: Could not match {0}!".format(ff))

## From the list of all file types, pick one at random

In [ ]:
scan = dict()
for section in scanable:
    scan[section] = dict()
    for ftype in scanable[section]:
        scan[section][ftype] = choice(scanable[section][ftype])
scan

## Run fitsverify on the chosen files

In [ ]:
for section in scan:
    for key in scan[section]:
        command = ['fitsverify', '-l', scan[section][key]]
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=specprod_path)
        out, err = proc.communicate()
        # print(section, key, out.decode('ascii').split('\n')[-2])
        result = out.decode('ascii').split('\n')[-2]
        if result == "**** Verification found 0 warning(s) and 0 error(s). ****":
            pass
            # print(section, key, "No problems detected.")
        else:
            print(section, key, "Problems detected!")
            print(out.decode('ascii'))
            if err:
                print(err.decode('ascii'))

## Generate datamodel stub files on all chosen files

In [ ]:
for section in scan:
    for key in scan[section]:
        command = ['generate_model', os.path.join(specprod_path, scan[section][key])]
        print(' '.join(command))
        proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE)
        out, err = proc.communicate()
        if proc.returncode != 0:
            print(out.decode('ascii'))
            print(err.decode('ascii'))
        if key == 'shifted-input-psf':
            src = 'shifted'
        elif key == 'fit-psf' or key == 'fit-psf-fixed-blacklisted' or key == 'fit-psf-before-blacklisted-fix':
            src = 'fit'
        elif key == 'exposure-qa':
            src = 'exposure'
        elif key == 'tile-qa':
            src = 'tile'
        else:
            src = key
        if os.path.exists(f'{src}.rst'):
            d = os.path.dirname(structure[section][key])
            if d and not os.path.isdir(d):
                print("os.makedirs('{0}')".format(d))
                os.makedirs(d)
            print("shutil.move('{0}.rst', '{1}'".format(src, structure[section][key]))
            shutil.move('{0}.rst'.format(src), structure[section][key])
        else:
            print("ERROR: Could not find file corresponding to {0}.rst -> {1}!".format(src, scan[section][key]))
            

## Scan preproc files for errors

In [ ]:
for p in choices(scanable['preproc']['preproc'], k=len(scanable['preproc']['preproc'])//100):
    command = ['fitsverify', '-l', p]
    proc = sp.Popen(command, stdout=sp.PIPE, stderr=sp.PIPE, cwd=specprod_path)
    out, err = proc.communicate()
    # print(section, key, out.decode('ascii').split('\n')[-2])
    result = out.decode('ascii').split('\n')[-2]
    if result == "**** Verification found 0 warning(s) and 0 error(s). ****":
        print(p, "No problems detected.")
    else:
        # print(p, "Problems detected!")
        o = out.decode('ascii')
        if ('*** Warning: Data checksum is not consistent with  the DATASUM keyword' in o and
            '*** Warning: HDU checksum is not in agreement with CHECKSUM.' in o):
            lines = o.split('\n')
            hduline = lines[lines.index('*** Warning: Data checksum is not consistent with  the DATASUM keyword') - 2]
            print(p, 'Checksum problems detected in {0}!'.format(hduline.strip('=').strip()))
            # print(out.decode('ascii'))
        else:
            print(p, "Other problems detected!")
        # if err:
        #     print(err.decode('ascii'))